[Bag of Words Meets Bags of Popcorn](https://www.kaggle.com/c/word2vec-nlp-tutorial/data)
======

## Data Set

The labeled data set consists of 50,000 IMDB movie reviews, specially selected for sentiment analysis. The sentiment of reviews is binary, meaning the IMDB rating < 5 results in a sentiment score of 0, and rating >=7 have a sentiment score of 1. No individual movie has more than 30 reviews. The 25,000 review labeled training set does not include any of the same movies as the 25,000 review test set. In addition, there are another 50,000 IMDB reviews provided without any rating labels.

## File descriptions

labeledTrainData - The labeled training set. The file is tab-delimited and has a header row followed by 25,000 rows containing an id, sentiment, and text for each review.
## Data fields

* id - Unique ID of each review
* sentiment - Sentiment of the review; 1 for positive reviews and 0 for negative reviews
* review - Text of the review

## Objective
Objective of this dataset is base on **review** we predict **sentiment** (positive or negative) so X is **review** column and y is **sentiment** column

## 1. Load Dataset
we only forcus on "labeledTrainData.csv" file

Let's first of all have a look at the data.

[Click here to download dataset](https://s3-ap-southeast-1.amazonaws.com/ml101-khanhnguyen/week3/assignment/labeledTrainData.tsv)

In [2]:
# Import pandas, numpy
import pandas as pd
import numpy as np


In [5]:
# Read dataset with extra params sep='\t', encoding="latin-1"
movie = pd.read_csv('./labeledTrainData.tsv', sep='\t', encoding='latin-1')
movie.head()

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...


## 2. Preprocessing

In [12]:
# stop words
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
stop = stopwords.words('english')


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/daotran/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [17]:
# Removing special characters and "trash"
import re
def preprocessor(text):
    # Remove HTML markup
    text = re.sub('<[^>]*>', '', text) # Your code here
    
    # Save emoticons for later appending
    # Your code here
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    
    # Remove any non-word character and append the emoticons,
    # removing the nose character for standarization. Convert to lower case
    # Your code here
    text = (re.sub('[\W]+', ' ', text.lower()) + ' ' + ' '.join(emoticons).replace('-', ''))
    
    return text

In [10]:
# tokenizer and stemming
# tokenizer: to break down our twits in individual words
# stemming: reducing a word to its root
from nltk.stem import PorterStemmer
# Your code here
porter = PorterStemmer()

def tokenizer_porter(text):
    token = [porter.stem(word) for word in text.split()]
    return token # Your code here

In [11]:
# split the dataset in train and test
# Your code here
from sklearn.model_selection import train_test_split
X = movie.review
y = movie.sentiment
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

## 3. Create Model and Train 

Using **Pipeline** to concat **tfidf** step and **LogisticRegression** step

In [18]:
# Import Pipeline, LogisticRegression, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words=stop, tokenizer=tokenizer_porter, 
                        preprocessor=preprocessor)# Your code here

clf = Pipeline([('vect', tfidf), ('clf', LogisticRegression(random_state=0))])
clf.fit(X_train, y_train)


Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2',
        preprocessor=<function preproc...nalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

## 4. Evaluate Model

In [25]:
# Using Test dataset to evaluate model
# classification_report
# confusion matrix
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

prediction = clf.predict(X_test)
print(classification_report(y_test, prediction))


             precision    recall  f1-score   support

          0       0.90      0.87      0.88      3796
          1       0.87      0.90      0.89      3704

avg / total       0.89      0.89      0.89      7500



In [59]:
twits = ['This film is so boring!', 'I have to go now', 'I don\'t like this movie',
         'It\'s so beautiful', 'I love you so much', 'This is really good', 'This is really bad', 
         'It\'s waste of time', 'I swear it is the worst I\'ve ever seen', 
         'Thanks god, It is the best I\'ve ever seen', 
         'It is the funniest I\'ve ever seen', 'Too many ads', 'It is not real']
preds = clf.predict_proba(twits)
for i in range(len(twits)):
    print(f'{twits[i]} --> Positive, Negative = {preds[i]}')

This film is so boring! --> Positive, Negative = [0.99314023 0.00685977]
I have to go now --> Positive, Negative = [0.60407386 0.39592614]
I don't like this movie --> Positive, Negative = [0.65014017 0.34985983]
It's so beautiful --> Positive, Negative = [0.02403512 0.97596488]
I love you so much --> Positive, Negative = [0.06504248 0.93495752]
This is really good --> Positive, Negative = [0.33455701 0.66544299]
This is really bad --> Positive, Negative = [0.99610592 0.00389408]
It's waste of time --> Positive, Negative = [0.98949256 0.01050744]
I swear it is the worst I've ever seen --> Positive, Negative = [0.95377839 0.04622161]
Thanks god, It is the best I've ever seen --> Positive, Negative = [0.06480358 0.93519642]
It is the funniest I've ever seen --> Positive, Negative = [0.08011975 0.91988025]
Too many ads --> Positive, Negative = [0.57187664 0.42812336]
It is not real --> Positive, Negative = [0.25297974 0.74702026]


## 5. Export Model 

In [66]:
# Using pickle to export our trained model
import pickle
import os

if not os.path.exists('data'):
    os.mkdir('data')
pickle.dump(clf, open(os.path.join('data', 'logisticRegression.pkl'), 'wb'), protocol=4)

In [68]:
loaded_model = pickle.load(open(os.path.join('data', 'logisticRegression.pkl'), 'rb'))

In [69]:
loaded_model.predict_proba(twits)

array([[0.99314023, 0.00685977],
       [0.60407386, 0.39592614],
       [0.65014017, 0.34985983],
       [0.02403512, 0.97596488],
       [0.06504248, 0.93495752],
       [0.33455701, 0.66544299],
       [0.99610592, 0.00389408],
       [0.98949256, 0.01050744],
       [0.95377839, 0.04622161],
       [0.06480358, 0.93519642],
       [0.08011975, 0.91988025],
       [0.57187664, 0.42812336],
       [0.25297974, 0.74702026]])